# Train with LightGBM in an Interactive Run

description: use mlflow for tracking local notebook experimentation in the cloud

## Pip install packages

When developing in a notebook, it is common to install packages. You can do this directly in a notebook:

In [ ]:
!pip install --upgrade numpy pandas scikit-learn lightgbm

## Setup cloud tracking

[Mlflow](https://github.com/mlflow/mlflow) is a great tool for local ML experimentation tracking. However, using it alone is like using git without GitHub. Your Azure Machine Learning workspace can easily be used to setup a remote tracking URI for mlflow:

In [ ]:
import mlflow
from azureml.core import Workspace

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment("lightgbm-iris-local-example")

## Load data

You can read directly from public URIs into Pandas. For private Blob or ADLS data, you can use built in Azure data protocols and pass in `storage_options` for credentials.

In [ ]:
data_uri = "https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"

In [ ]:
import pandas as pd

df = pd.read_csv(data_uri)
df.head()

## Write functions

After some experimentation, you may refactor your code into a few functions for logical steps in the ML training process:

In [ ]:
# imports
import time

import lightgbm as lgb

from sklearn.metrics import log_loss, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# define functions
def preprocess_data(df):
    X = df.drop(["species"], axis=1)
    y = df["species"]

    enc = LabelEncoder()
    y = enc.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    return X_train, X_test, y_train, y_test, enc


def train_model(params, num_boost_round, X_train, X_test, y_train, y_test):
    t1 = time.time()
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test)
    model = lgb.train(
        params,
        train_data,
        num_boost_round=num_boost_round,
        valid_sets=[test_data],
        valid_names=["test"],
    )
    t2 = time.time()

    return model, t2 - t1


def evaluate_model(model, X_test, y_test):
    y_proba = model.predict(X_test)
    y_pred = y_proba.argmax(axis=1)
    loss = log_loss(y_test, y_proba)
    acc = accuracy_score(y_test, y_pred)

    return loss, acc

## Run a trial

Now, you can easily run local trials, editing the parameters and seeing how the model performs.

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

# preprocess data
X_train, X_test, y_train, y_test, enc = preprocess_data(df)

# set training parameters
params = {
    "objective": "multiclass",
    "num_class": 3,
    "learning_rate": 0.1,
    "metric": "multi_logloss",
    "colsample_bytree": 1.0,
    "subsample": 1.0,
    "seed": 42,
}

num_boost_round = 32

# start run
run = mlflow.start_run()

# enable automatic logging
mlflow.lightgbm.autolog()

# train model
model, train_time = train_model(
    params, num_boost_round, X_train, X_test, y_train, y_test
)
mlflow.log_metric("training_time", train_time)

# evaluate model
loss, acc = evaluate_model(model, X_test, y_test)
mlflow.log_metrics({"loss": loss, "accuracy": acc})

# end run
mlflow.end_run()